In [46]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [47]:
#Read the csv files and store them in a dataframe
m1_6v = pd.read_csv('data/Motor1-6v.csv')
m1_8v = pd.read_csv('data/Motor1-8v.csv')
m1_10v = pd.read_csv('data/Motor1-10v.csv')
m1_12v = pd.read_csv('data/Motor1-12v.csv')
m2_6v = pd.read_csv('data/Motor2-6v.csv')
m2_8v = pd.read_csv('data/Motor2-8v.csv')
m2_10v = pd.read_csv('data/Motor2-10v.csv')
m2_12v = pd.read_csv('data/Motor2-12v.csv')
m3_6v = pd.read_csv('data/Motor3-6v.csv')
m3_8v = pd.read_csv('data/Motor3-8v.csv')
m3_10v = pd.read_csv('data/Motor3-10v.csv')
m3_12v = pd.read_csv('data/Motor3-12v.csv')

m1 = pd.concat([m1_6v, m1_8v, m1_10v, m1_12v], ignore_index=True)
m2 = pd.concat([m2_6v, m2_8v, m2_10v, m2_12v], ignore_index=True)
m3 = pd.concat([m3_6v, m3_8v, m3_10v, m3_12v], ignore_index=True)

def thrust_model(pwm_signal, battery_voltage):
  """
  This function calculates the thrust based on PWM signal and battery voltage
  using a multiple linear regression model.

  Args:
      pwm_signal: PWM signal value (0-117)
      battery_voltage: Battery voltage (V)

  Returns:
      The predicted thrust (N)
  """

  # Model coefficients obtained from linear regression
  intercept = -3.2800771299179194
  coefficients = [0.03788209, 0.30441551]

  # Apply the model equation
  thrust = intercept + coefficients[0] * pwm_signal + coefficients[1] * battery_voltage
  return thrust

In [48]:
data1 = m1.copy()
data2 = m2.copy()
data3 = m3.copy()

# print column names in data
print(data1.columns)

Index(['PWM_Strength', 'Voltage', 'Weight1(g)'], dtype='object')


In [80]:
from sklearn.linear_model import LinearRegression

def thrust_model(data, plot=False, motor_number=9):
    # for m1 create column with pwm^2
    data['PWM^2'] = data['PWM_Strength']**2
    # create column with thrust in newtons instead of grams
    data['Thrust'] = (data['Weight1(g)']*-1) * 0.00980665

    # Separate features (X) and target variable (y)
    X = data[['PWM^2', 'Voltage']]
    y = data['Thrust']

    # Create and fit the model
    model = LinearRegression(positive=True)
    model.fit(X, y)

    # Print the coefficients and intercept
    print('Intercept:', model.intercept_)
    print('Coefficients:', model.coef_)

    b = model.intercept_
    a1 = model.coef_[0]
    a2 = model.coef_[1]

    if plot == True:
        # draw a 3d scatter plot of the data points and the regression plane
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(data['PWM^2'], data['Voltage'], data['Thrust'], c='r', marker='o')
        # draw the regression plane
        x = np.linspace(data['PWM^2'].min(), data['PWM^2'].max(), 100)
        y = np.linspace(data['Voltage'].min(), data['Voltage'].max(), 100)
        x, y = np.meshgrid(x, y)
        z = model.intercept_ + model.coef_[0] * x + model.coef_[1] * y
        ax.plot_surface(x, y, z, alpha=0.5)

        ax.set_xlabel('PWM^2')
        ax.set_ylabel('Voltage')
        ax.set_zlabel('Thrust')
        ax.set_title('Motor ' + str(motor_number))
        plt.show()

    return b, a1, a2

In [81]:
m1_b, m1_a1, m1_a2 = thrust_model(data1, motor_number=1)
m2_b, m2_a1, m2_a2 = thrust_model(data2, motor_number=2)
m3_b, m3_a1, m3_a2 = thrust_model(data3, motor_number=3)

Intercept: -1.5519274278989248
Coefficients: [0.00022836 0.20206462]
Intercept: -1.3905525531417564
Coefficients: [0.00022609 0.18306283]
Intercept: -1.413300451227128
Coefficients: [0.00022565 0.18582327]


In [82]:
pwm = 1
voltage = 10

thrust_1 = m1_b + m1_a1 * pwm**2 + m1_a2 * voltage
thrust_2 = m2_b + m2_a1 * pwm**2 + m2_a2 * voltage
thrust_3 = m3_b + m3_a1 * pwm**2 + m3_a2 * voltage

print('Thrust for motor 1:', thrust_1)
print('Thrust for motor 2:', thrust_2)
print('Thrust for motor 3:', thrust_3)
print()
print("Thrust equations:")
print("thrust_m1 = {:.9f} + {:.9f} * pwm**2 + {:.9f} * voltage".format(m1_b, m1_a1, m1_a2))
print("thrust_m2 = {:.9f} + {:.9f} * pwm**2 + {:.9f} * voltage".format(m2_b, m2_a1, m2_a2))
print("thrust_m3 = {:.9f} + {:.9f} * pwm**2 + {:.9f} * voltage".format(m3_b, m3_a1, m3_a2))

Thrust for motor 1: 0.46894717399935004
Thrust for motor 2: 0.4403018472465965
Thrust for motor 3: 0.44515791682675965

Thrust equations:
thrust_m1 = -1.551927428 + 0.000228365 * pwm**2 + 0.202064624 * voltage
thrust_m2 = -1.390552553 + 0.000226092 * pwm**2 + 0.183062831 * voltage
thrust_m3 = -1.413300451 + 0.000225653 * pwm**2 + 0.185823271 * voltage


In [52]:
thrust_m1 = -1.5519274278989252 + 0.00022836 * pwm**2 + 0.20206462 * voltage
thrust_m2 = -1.3905525531417573 + 0.00022609 * pwm**2 + 0.18306283 * voltage
thrust_m3 = -1.4133004512271263 + 0.00022565 * pwm**2 + 0.18582327 * voltage

In [ ]:
from sklearn.linear_model import PolynomialFeatures

def thrust_model(data, plot=False, motor_number=9):
    # for m1 create column with pwm^2
    data['PWM^2'] = data['PWM_Strength']**2
    # create column with thrust in newtons instead of grams
    data['Thrust'] = (data['Weight1(g)']*-1) * 0.00980665

    # Separate features (X) and target variable (y)
    X = data[['PWM^2', 'Voltage']]
    y = data['Thrust']

    # Create and fit the model
    model = PolynomialFeatures()
    model.fit(X, y)

    # Print the coefficients
    print('Intercept:', model.intercept_)
    print('Coefficients:', model.coef_)

    b = model.intercept_
    a1 = model.coef_[0]
    a2 = model.coef_[1]

    if plot == True:
        # draw a 3d scatter plot of the data points and the regression plane
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(data['PWM^2'], data['Voltage'], data['Thrust'], c='r', marker='o')
        # draw the regression plane
        x = np.linspace(data['PWM^2'].min(), data['PWM^2'].max(), 100)
        y = np.linspace(data['Voltage'].min(), data['Voltage'].max(), 100)
        x, y = np.meshgrid(x, y)
        z = model.intercept_ + model.coef_[0] * x + model.coef_[1] * y
        ax.plot_surface(x, y, z, alpha=0.5)

        ax.set_xlabel('PWM^2')
        ax.set_ylabel('Voltage')
        ax.set_zlabel('Thrust')
        ax.set_title('Motor ' + str(motor_number))
        plt.show()

    return b, a1, a2

In [85]:

data = m1.copy()

# create column with pwm^2
data['PWM^2'] = data['PWM_Strength']**2
# create column with thrust in newtons instead of grams
data['Thrust'] = (data['Weight1(g)']*-1) * 0.00980665

# air density
rho = 1.225

# propeller diameter
D = 0.2032

# thrust coefficient
kt = 6.769e-6

# create new column with calculated RPM values from thrust coefficient, air density, propeller diameter and thrust
data['RPS'] = data['Thrust']**(1/2)/(D**2*kt**(1/2)*rho**(1/2))
data['RPM'] = data['RPS']*60

In [87]:
data.head(20)


,PWM_Strength,Voltage,Weight1(g),PWM^2,Thrust,RPS,RPM
0,0.0,6.00,-0.46,0.0,0.004511,564.886954,33893.217251
1,1.0,5.98,-0.41,1.0,0.004021,533.303652,31998.219108
2,2.0,5.98,-0.39,4.0,0.003825,520.133628,31208.017660
3,3.0,5.98,-0.37,9.0,0.003628,506.621354,30397.281235
4,4.0,6.00,-0.42,16.0,0.004119,539.768175,32386.090471
5,5.0,5.98,-0.37,25.0,0.003628,506.621354,30397.281235
6,6.0,5.98,-0.52,36.0,0.005099,600.598580,36035.914794
7,7.0,6.00,-0.52,49.0,0.005099,600.598580,36035.914794
8,8.0,5.98,-0.51,64.0,0.005001,594.795559,35687.733539
9,9.0,5.98,-0.71,81.0,0.006963,701.797479,42107.848713


In [88]:
data.tail(20)

,PWM_Strength,Voltage,Weight1(g),PWM^2,Thrust,RPS,RPM
470,102.0,10.21,-375.16,10404.0,3.679063,16132.099528,9.679260e+05
471,103.0,12.69,-379.26,10609.0,3.719270,16220.011191,9.732007e+05
472,104.0,12.41,-382.92,10816.0,3.755162,16298.087845,9.778853e+05
473,105.0,12.45,-386.36,11025.0,3.788897,16371.131911,9.822679e+05
474,106.0,9.87,-388.44,11236.0,3.809295,16415.140410,9.849084e+05
475,107.0,10.13,-393.11,11449.0,3.855092,16513.520692,9.908112e+05
476,108.0,10.89,-396.00,11664.0,3.883433,16574.110201,9.944466e+05
477,109.0,12.49,-407.30,11881.0,3.994249,16808.920928,1.008535e+06
478,110.0,11.05,-405.93,12100.0,3.980813,16780.627755,1.006838e+06
479,111.0,12.32,-408.73,12321.0,4.008272,16838.402510,1.010304e+06
